In [2]:
import cv2
import numpy as np
from pathlib import Path

In [3]:
root = Path().resolve().parent.parent

In [4]:
feature_path = root / 'datasets' / 'ccd' / 'vgg16_features'
file_class = 'positive'
np_file = '000004.npz'

In [5]:
np_feature = np.load(feature_path / file_class / np_file)

In [6]:
np_feature['data'].shape

(50, 20, 4096)

In [7]:
np_feature

NpzFile '/home/don/Projects/misc/major-project-works/datasets/ccd/vgg16_features/positive/000004.npz' with keys: data, det, labels, ID

In [8]:
np_feature['det'].shape

(50, 19, 6)

In [9]:
video_path = root / 'datasets' / 'ccd' /'videos'
videofile = '000004.mp4'

In [10]:
cap = cv2.VideoCapture(str(video_path / file_class / videofile))

In [11]:
frame_index = 0
fps = 10  # Video frame rate

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("End of video reached or error reading frame.")
        break

    # Draw bounding boxes for the current frame
    if frame_index < len(np_feature['det']):
        for det in np_feature['det'][frame_index]:
            x1, y1, x2, y2, prob, cls = det

            color = (0, 255, 0) if cls == 1 else (0, 0, 255)  # Green for class 1, red otherwise
            thickness = 2
            frame = cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, thickness)
            label = f"Class: {int(cls)}, Prob: {prob:.2f}"
            frame = cv2.putText(frame, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

    # Display the frame
    cv2.imshow("Video with Bounding Boxes", frame)

    # Wait for the correct time to match frame rate
    key = cv2.waitKey(0)  # Wait indefinitely for keypress
    if key & 0xFF == ord('q'):  # Quit on 'q'
        break
    elif key & 0xFF == ord('n'):  # Next frame on 'n'
        frame_index += 1
    elif key & 0xFF == ord('p'):  # Previous frame on 'p'
        frame_index = max(0, frame_index - 1)
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index)

cap.release()
#cv2.destroyAllWindows()